In [ ]:
!pip install jira

In [ ]:
from jira import JIRA
import re
import numpy as np

In [ ]:
print('init')
jira = JIRA('https://support.subshell.com',basic_auth=(username, password))
print(jira)
issues = []
start = 0
size = 100
do_next = True
while do_next:
    do_next = False
    print('loop')
    search = jira.search_issues('cf[10580] = Weasel and status in ("Gelöst / Resolved", "Geschlossen / Closed") and Sprint is not EMPTY',
                               startAt= start, size = size)
    print('Search', start, size)
    do_next = len(search) == size
    start = start + size
    
    for issue_found in search:
        issue = {}
    
    
        sprints =  getattr(issue_found.fields, 'customfield_10880')
        sprints = [re.findall(r'Weasel \d+', sprint) for sprint in sprints]
        sprints = np.array([np.array(x) for x in sprints if len(x) > 0]).flatten()
        timeSpent = 0
        for worklog in jira.worklogs(issue_found):
            timeSpent = timeSpent + worklog.timeSpentSeconds
        originalEstimate = issue_found.fields.__dict__.get('timeoriginalestimate')
    
        issue['sprints'] = sprints
        issue['timeSpent'] = timeSpent
        issue['originalEstimate'] = originalEstimate
        issue['key'] = issue_found.key
        issue['project'] = issue_found.fields.project.key
        issue['issueType'] = getattr(issue_found.fields, 'issuetype').name
        issue['title'] = getattr(issue_found.fields, 'summary')
        # print(issue)
        # print('--')
        issues.append(issue)
    #print(issue_found.key, getattr(issue_found.fields, 'customfield_10880'))

In [ ]:
import json

with open('issues_raw.json', 'w') as file:
    json.dump(issues, file)